<a href="https://colab.research.google.com/github/Ajay07pandey/DOI-Number-To-PDF/blob/main/DOI_To_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

def download_pdf(base_url, doi_number, title):
    website_link = urljoin(base_url, doi_number)
    save_folder = "/content/New folder"

    try:
        response = requests.get(website_link)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            button_tag = soup.find('div', id='buttons').find('button')
        except AttributeError:
            print(f"This DOI not present in scihub: {doi_number}")
            return

        if button_tag and 'onclick' in button_tag.attrs:
            onclick_attr = button_tag['onclick']
            pdf_url = urljoin(website_link, onclick_attr.split("'")[1])

            pdf_response = requests.get(pdf_url)
            pdf_response.raise_for_status()

            filename = f"{title}.pdf"
            save_path = os.path.join(save_folder, filename)

            with open(save_path, "wb") as f:
                f.write(pdf_response.content)
            print(f"PDF downloaded successfully: {filename}")
        else:
            print(f"Unable to download {doi_number}")
    except requests.RequestException as e:
        print(f"Error downloading PDF for DOI {doi_number}: {e}")

def main(input_filename):
    # Read the Excel file
    df = pd.read_excel(input_filename)

    # Download PDFs for each DOI number with corresponding file names
    base_url = "https://sci-hub.se/"
    for index, row in df.iterrows():
        doi = str(row['DOI'])
        title = str(row['Title'])
        download_pdf(base_url, doi, title)

if __name__ == "__main__":
    # Specify the input file
    input_filename = "/content/Doi_number_file.xlsx"  # Change this to your input Excel file

    # Run the main function
    main(input_filename)
